In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

# 랜덤 시드 고정 (재현 가능한 결과)
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# GPU 메모리 증분 할당 (메모리 부족 방지)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU 설정 완료: {len(gpus)}개 GPU 발견")
    except RuntimeError as e:
        print(f"GPU 설정 오류: {e}")
else:
    print("⚠️ GPU를 찾을 수 없습니다. CPU로 실행됩니다.")

print("✅ 환경 설정 완료")

⚠️ GPU를 찾을 수 없습니다. CPU로 실행됩니다.
✅ 환경 설정 완료


In [3]:
# 데이터 경로 설정
base_path = r'D:\my_projects\calmman-facial-classification\data\processed'
teasing_path = os.path.join(base_path, 'teasing')
non_teasing_path = os.path.join(base_path, 'non_teasing')

# 클래스 정의 (이진분류)
classes = ['non_teasing', 'teasing']  # 0: 약올리지 않음, 1: 약올림
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

print(f"클래스 매핑: {class_to_index}")
print(f"약올리기 폴더 존재: {os.path.exists(teasing_path)}")
print(f"비약올리기 폴더 존재: {os.path.exists(non_teasing_path)}")

# 이미지 확장자 정의
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

클래스 매핑: {'non_teasing': 0, 'teasing': 1}
약올리기 폴더 존재: True
비약올리기 폴더 존재: True


In [14]:
import cv2
import numpy as np
from PIL import Image

def load_images_robust(folder_path, label, max_images=None):
    """강화된 이미지 로딩 함수"""
    images = []
    labels = []
    failed_files = []
    
    if not os.path.exists(folder_path):
        print(f"경고: {folder_path} 폴더가 존재하지 않습니다.")
        return images, labels, failed_files
    
    file_list = [f for f in os.listdir(folder_path) if f.lower().endswith(image_extensions)]
    
    # 파일 수 제한 (메모리 절약)
    if max_images:
        file_list = file_list[:max_images]
    
    for fname in tqdm(file_list, desc=f"Loading {os.path.basename(folder_path)}"):
        img_path = os.path.join(folder_path, fname)
        
        try:
            # OpenCV로 시도
            img = cv2.imread(img_path)
            
            if img is None:
                # PIL로 재시도 (한글 경로 문제 해결)
                pil_img = Image.open(img_path)
                img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
            
            if img is None or img.shape[0] == 0 or img.shape[1] == 0:
                raise Exception("빈 이미지")
            
            # BGR to RGB 변환
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # 크기 조정
            img = cv2.resize(img, (224, 224))
            
            # 정규화
            img = img.astype('float32') / 255.0
            
            images.append(img)
            labels.append(label)
            
        except Exception as e:
            failed_files.append(fname)
            continue
    
    return images, labels, failed_files

# 데이터 로딩
print("=== 데이터 로딩 시작 ===")

# 비약올리기 이미지 로드 (라벨: 0)
X_non_teasing, y_non_teasing, failed_non_teasing = load_images_robust(non_teasing_path, 0)

# 약올리기 이미지 로드 (라벨: 1)
X_teasing, y_teasing, failed_teasing = load_images_robust(teasing_path, 1)

# 데이터 합치기
X = X_non_teasing + X_teasing
y = y_non_teasing + y_teasing

print(f"\n=== 로딩 결과 ===")
print(f"총 이미지 수: {len(X)}")
print(f"비약올리기 이미지: {len(X_non_teasing)}")
print(f"약올리기 이미지: {len(X_teasing)}")
print(f"실패한 파일: {len(failed_non_teasing) + len(failed_teasing)}개")

if len(X) == 0:
    print("❌ 로드된 이미지가 없습니다. 경로를 확인하세요.")
else:
    print("✅ 데이터 로딩 완료")

=== 데이터 로딩 시작 ===


Loading teasing: 100%|██████████| 78/78 [00:00<00:00, 973.55it/s]


=== 로딩 결과 ===
총 이미지 수: 204
비약올리기 이미지: 126
약올리기 이미지: 78
실패한 파일: 0개
✅ 데이터 로딩 완료


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_both_classes(X_list, y_list, target_per_class=250):
    """양쪽 클래스 모두 목표 개수까지 증강"""
    
    # 클래스별 데이터 분리
    class_0_data = [X_list[i] for i in range(len(X_list)) if y_list[i] == 0]
    class_1_data = [X_list[i] for i in range(len(X_list)) if y_list[i] == 1]
    
    print(f"증강 전:")
    print(f"  비약올리기: {len(class_0_data)}개")
    print(f"  약올리기: {len(class_1_data)}개")
    
    # 증강기 설정 (양쪽 동일한 품질)
    datagen = ImageDataGenerator(
        # rotation_range=20,         # 제거!
        width_shift_range=0.1,       # 최소한으로
        height_shift_range=0.1,      
        horizontal_flip=True,        # OK (좌우 대칭)
        zoom_range=0.1,              # 최소한으로
        brightness_range=[0.8, 1.2], # 조명만 변경
        fill_mode='nearest'
    )
    
    final_class_0 = class_0_data.copy()
    final_class_1 = class_1_data.copy()
    
    # 비약올리기 클래스 증강
    if len(class_0_data) < target_per_class:
        need_count_0 = target_per_class - len(class_0_data)
        print(f"비약올리기 증강: {need_count_0}개 생성")
        
        for i in tqdm(range(need_count_0), desc="비약올리기 증강"):
            base_img = class_0_data[i % len(class_0_data)]
            img_batch = np.expand_dims(base_img, 0)
            aug_iter = datagen.flow(img_batch, batch_size=1)
            aug_img = next(aug_iter)[0]
            aug_img = np.clip(aug_img, 0.0, 1.0)
            final_class_0.append(aug_img)
    
    # 약올리기 클래스 증강
    if len(class_1_data) < target_per_class:
        need_count_1 = target_per_class - len(class_1_data)
        print(f"약올리기 증강: {need_count_1}개 생성")
        
        for i in tqdm(range(need_count_1), desc="약올리기 증강"):
            base_img = class_1_data[i % len(class_1_data)]
            img_batch = np.expand_dims(base_img, 0)
            aug_iter = datagen.flow(img_batch, batch_size=1)
            aug_img = next(aug_iter)[0]
            aug_img = np.clip(aug_img, 0.0, 1.0)
            final_class_1.append(aug_img)
    
    # 최종 데이터 구성
    final_X = final_class_0 + final_class_1
    final_y = [0] * len(final_class_0) + [1] * len(final_class_1)
    
    print(f"\n증강 후:")
    print(f"  비약올리기: {len(final_class_0)}개")
    print(f"  약올리기: {len(final_class_1)}개")
    print(f"  총 이미지: {len(final_X)}개")
    print(f"  완벽한 균형: {len(final_class_1)/len(final_class_0):.3f}")
    
    return final_X, final_y

# 양쪽 클래스 모두 증강 (각각 250개씩)
if len(X) > 0:
    X, y = augment_both_classes(X, y, target_per_class=250)
    print("✅ 양쪽 클래스 증강 완료")
else:
    print("❌ 증강할 데이터가 없습니다.")

증강 전:
  비약올리기: 126개
  약올리기: 78개
비약올리기 증강: 124개 생성


비약올리기 증강: 100%|██████████| 124/124 [00:00<00:00, 258.31it/s]


약올리기 증강: 172개 생성


약올리기 증강: 100%|██████████| 172/172 [00:00<00:00, 273.72it/s]


증강 후:
  비약올리기: 250개
  약올리기: 250개
  총 이미지: 500개
  완벽한 균형: 1.000
✅ 양쪽 클래스 증강 완료


In [16]:
# NumPy 배열 변환
X = np.array(X)
y = np.array(y)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"클래스 분포: {np.bincount(y)}")

# 계층적 분할 (클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

print(f"\n=== 데이터 분할 결과 ===")
print(f"훈련 데이터: {X_train.shape}, 레이블: {y_train.shape}")
print(f"검증 데이터: {X_val.shape}, 레이블: {y_val.shape}")
print(f"훈련 클래스 분포: {np.bincount(y_train)}")
print(f"검증 클래스 분포: {np.bincount(y_val)}")

# 클래스 가중치 미리 계산
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {
    0: class_weights[0], 
    1: class_weights[1] 
}

print(f"클래스 가중치: {class_weight_dict}")
print("✅ 데이터 준비 완료")

X shape: (500, 224, 224, 3)
y shape: (500,)
클래스 분포: [250 250]

=== 데이터 분할 결과 ===
훈련 데이터: (400, 224, 224, 3), 레이블: (400,)
검증 데이터: (100, 224, 224, 3), 레이블: (100,)
훈련 클래스 분포: [200 200]
검증 클래스 분포: [50 50]
클래스 가중치: {0: 1.0, 1: 1.0}
✅ 데이터 준비 완료


In [11]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

print("=== 🚀 시동 강화 모델 구성 ===")

# EfficientNetB0 백본 로드 및 동결
base_model = EfficientNetB0(
    include_top=False, 
    weights='imagenet', 
    input_shape=(224, 224, 3)
)
base_model.trainable = False
print("✅ EfficientNetB0 백본 로드 완료 (동결됨)")

# 분류기 헤드 구성
x = GlobalAveragePooling2D(name='feature_pooling')(base_model.output)
x = Dropout(0.3, name='dropout_1')(x)
x = Dense(128, activation='relu', name='dense_128')(x)
x = Dropout(0.2, name='dropout_2')(x)
output = Dense(1, activation='sigmoid', name='classification_head')(x)

model = Model(inputs=base_model.input, outputs=output, name='teasing_classifier')

print("✅ 모델 아키텍처 구성 완료")

# 🔧 시동 걸기 1: 가중치 강제 초기화
print("\n🔧 시동 걸기 1단계: 가중치 강제 초기화")

# 최종 Dense 레이어 찾기
final_layer = None
dense_128_layer = None

for layer in model.layers:
    if layer.name == 'classification_head':
        final_layer = layer
    elif layer.name == 'dense_128':
        dense_128_layer = layer

if final_layer:
    # 최종 레이어 가중치 초기화 (더 큰 분산)
    weights, bias = final_layer.get_weights()
    new_weights = np.random.normal(0, 0.6, weights.shape)  # 표준편차 0.6
    new_bias = np.array([0.0])
    final_layer.set_weights([new_weights, new_bias])
    print(f"   최종 레이어 가중치 초기화: 평균={new_weights.mean():.6f}, 표준편차={new_weights.std():.6f}")

if dense_128_layer:
    # Dense(128) 레이어도 강화 초기화
    weights, bias = dense_128_layer.get_weights()
    new_weights = np.random.normal(0, 0.1, weights.shape)  # He 초기화 기반
    new_bias = np.zeros_like(bias)
    dense_128_layer.set_weights([new_weights, new_bias])
    print(f"   Dense(128) 레이어 가중치 초기화 완료")

# 🔧 시동 걸기 2: 높은 학습률로 컴파일
print("\n🔧 시동 걸기 2단계: 높은 학습률 설정")
model.compile(
    optimizer=Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999),  # 높은 학습률
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(f"   학습률: 0.01 (기본값의 10배)")
print("✅ 시동 강화 모델 준비 완료!")

# 모델 요약
print(f"\n=== 모델 요약 ===")
print(f"총 파라미터: {model.count_params():,}")
print(f"훈련 가능 파라미터: {sum([tf.size(var).numpy() for var in model.trainable_variables]):,}")

=== 🚀 시동 강화 모델 구성 ===
✅ EfficientNetB0 백본 로드 완료 (동결됨)
✅ 모델 아키텍처 구성 완료

🔧 시동 걸기 1단계: 가중치 강제 초기화
   최종 레이어 가중치 초기화: 평균=0.002313, 표준편차=0.578500
   Dense(128) 레이어 가중치 초기화 완료

🔧 시동 걸기 2단계: 높은 학습률 설정
   학습률: 0.01 (기본값의 10배)
✅ 시동 강화 모델 준비 완료!

=== 모델 요약 ===
총 파라미터: 4,213,668
훈련 가능 파라미터: 164,097


In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# 강화된 콜백 설정
callbacks = [
    # 조기 종료 (더 인내심 있게)
    EarlyStopping(
        monitor='val_loss', 
        patience=8, 
        restore_best_weights=True,
        verbose=1
    ),
    
    # 최고 모델 저장
    ModelCheckpoint(
        'best_teasing_model_enhanced.h5', 
        monitor='val_accuracy', 
        save_best_only=True,
        verbose=1
    ),
    
    # 학습률 감소 (플래토 상황에서)
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-6,
        verbose=1
    )
]

print("✅ 강화된 콜백 설정 완료")

✅ 강화된 콜백 설정 완료


In [13]:
print("=== 🚀 3단계 학습 프로세스 시작 ===")

# === 1단계: 시동 걸기 학습 ===
print("\n🔥 1단계: 시동 걸기 (강력한 학습)")
history_kickstart = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=8,  # 작은 배치로 자주 업데이트
    class_weight=class_weight_dict,
    verbose=1
)

# 시동 상태 확인
val_loss_1, val_acc_1 = model.evaluate(X_val, y_val, verbose=0)
y_pred_1 = model.predict(X_val, verbose=0)
pred_std_1 = y_pred_1.std()

print(f"\n1단계 결과: 정확도={val_acc_1:.4f}, 예측 다양성={pred_std_1:.4f}")

if pred_std_1 < 0.05:
    print("⚠️ 시동이 완전히 걸리지 않음. 추가 조치 필요...")
    
    # 극단적 클래스 가중치로 재시도
    extreme_weights = {0: 1.0, 1: 15.0}  # 15배 가중치
    print("🔥 극단적 클래스 가중치로 재학습...")
    
    history_extreme = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=4,  # 더 작은 배치
        class_weight=extreme_weights,
        verbose=1
    )
    
    # 재확인
    y_pred_1 = model.predict(X_val, verbose=0)
    pred_std_1 = y_pred_1.std()
    print(f"극단적 가중치 후 예측 다양성: {pred_std_1:.4f}")

# === 2단계: 학습률 조정 및 안정화 ===
print(f"\n🎯 2단계: 학습률 조정 및 안정화")
model.compile(
    optimizer=Adam(learning_rate=0.003),  # 중간 학습률
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_stable = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=16,
    class_weight=class_weight_dict,
    callbacks=[callbacks[2]],  # ReduceLROnPlateau만 사용
    verbose=1
)

# === 3단계: 미세 조정 ===
print(f"\n✨ 3단계: 미세 조정")
model.compile(
    optimizer=Adam(learning_rate=0.0005),  # 낮은 학습률
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=callbacks,  # 모든 콜백 사용
    verbose=1
)

# 전체 히스토리 합치기
history = type('History', (), {})()
history.history = {}

all_histories = [history_kickstart, history_stable, history_fine]
if 'history_extreme' in locals():
    all_histories.insert(1, history_extreme)

for key in history_kickstart.history.keys():
    history.history[key] = []
    for hist in all_histories:
        if key in hist.history:
            history.history[key].extend(hist.history[key])

print("✅ 3단계 학습 완료!")

=== 🚀 3단계 학습 프로세스 시작 ===

🔥 1단계: 시동 걸기 (강력한 학습)
Epoch 1/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 108ms/step - accuracy: 0.5880 - loss: 6.6009 - val_accuracy: 0.5000 - val_loss: 1.2730
Epoch 2/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.5525 - loss: 1.5757 - val_accuracy: 0.5000 - val_loss: 0.7271
Epoch 3/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.5583 - loss: 1.0100 - val_accuracy: 0.5000 - val_loss: 0.7443
Epoch 4/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.5677 - loss: 0.9697 - val_accuracy: 0.5000 - val_loss: 0.7167
Epoch 5/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - accuracy: 0.5677 - loss: 0.9803 - val_accuracy: 0.5000 - val_loss: 0.7109
Epoch 6/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - accuracy: 0.5677 - loss: 0.9932 - val_accuracy: 0.5000 - val_loss: 0.7157
Epoch 7/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 94ms/step - accuracy: 0.5677 - loss: 0.9779 - val_accuracy: 0.5000 - val_loss: 0.7219
Epoch 8/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 94ms/step - accuracy:

13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 422ms/step - accuracy: 0.5553 - loss: 0.9593 - val_accuracy: 0.5000 - val_loss: 0.8077 - learning_rate: 5.0000e-04
Epoch 2/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - accuracy: 0.5595 - loss: 0.9792
Epoch 2: val_accuracy did not improve from 0.50000
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 303ms/step - accuracy: 0.5553 - loss: 0.9808 - val_accuracy: 0.5000 - val_loss: 0.8073 - learning_rate: 5.0000e-04
Epoch 3/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.5595 - loss: 1.0277
Epoch 3: val_accuracy did not improve from 0.50000
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 280ms/step - accuracy: 0.5553 - loss: 1.0289 - val_accuracy: 0.5000 - val_loss: 0.8106 - learning_rate: 5.0000e-04
Epoch 4/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.5595 - loss: 0.9641
Epoch 4: val_accuracy did not improve from 0.50000
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 281ms/step - accuracy: 0.5553 - loss: 0.9686 - val_accuracy: 0.5000 - val_loss: 0.8092 - learning_rate: 5.0000e-04
Epoch 5/

KeyboardInterrupt: 